In [25]:
#importing library
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [26]:
#importing the data
df = pd.read_csv('M1_final.csv')
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,DEST,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_M,DEP_TIME_M,CRS_ARR_M,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,sch_dep,sch_arr,TAXI_OUT
0,11,1,5,B6,N828JB,CHS,-1,124,636,324,323,448,48,34,58,W,25,38,29.86,Fair / Windy,9,17,14
1,11,1,5,B6,N992JB,LAX,-7,371,2475,340,333,531,48,34,58,W,25,38,29.86,Fair / Windy,9,17,15
2,11,1,5,B6,N959JB,FLL,40,181,1069,301,341,482,48,34,58,W,25,38,29.86,Fair / Windy,9,17,22
3,11,1,5,B6,N999JQ,MCO,-2,168,944,345,343,513,48,34,58,W,25,38,29.86,Fair / Windy,9,17,12
4,11,1,5,DL,N880DN,ATL,-4,139,760,360,356,499,46,32,58,W,24,35,29.91,Fair / Windy,9,17,13


In [27]:
#Replacing the missing value with mode value as the column is of dtype = object
df['Wind'] = df['Wind'].fillna('W')

In [28]:
df['Dew Point'] = df['Dew Point'].astype('int')

In [29]:
#not considering tail_num as it has many unique values
df=df.drop(['TAIL_NUM'],axis=1)
cat_col_one_hot = ['OP_UNIQUE_CARRIER','DEST','Wind','Condition']
for i in cat_col_one_hot:
    df[i] = df[i].astype('category')

In [30]:
df=pd.get_dummies(df,columns=cat_col_one_hot,drop_first=True)
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_M,DEP_TIME_M,CRS_ARR_M,Temperature,Dew Point,Humidity,Wind Speed,Wind Gust,Pressure,sch_dep,sch_arr,TAXI_OUT,OP_UNIQUE_CARRIER_AA,OP_UNIQUE_CARRIER_AS,OP_UNIQUE_CARRIER_B6,OP_UNIQUE_CARRIER_DL,OP_UNIQUE_CARRIER_HA,OP_UNIQUE_CARRIER_MQ,OP_UNIQUE_CARRIER_OO,OP_UNIQUE_CARRIER_YX,DEST_ATL,DEST_AUS,DEST_BNA,DEST_BOS,DEST_BQN,DEST_BTV,DEST_BUF,DEST_BUR,DEST_BWI,DEST_CHS,DEST_CLE,DEST_CLT,DEST_CMH,DEST_CVG,...,Wind_ENE,Wind_ESE,Wind_N,Wind_NE,Wind_NNE,Wind_NNW,Wind_NW,Wind_S,Wind_SE,Wind_SSE,Wind_SSW,Wind_SW,Wind_VAR,Wind_W,Wind_WNW,Wind_WSW,Condition_Cloudy / Windy,Condition_Drizzle and Fog,Condition_Fair,Condition_Fair / Windy,Condition_Fog,Condition_Fog / Windy,Condition_Heavy Rain,Condition_Heavy Rain / Windy,Condition_Light Drizzle,Condition_Light Drizzle / Windy,Condition_Light Freezing Rain,Condition_Light Rain,Condition_Light Rain / Windy,Condition_Light Snow,Condition_Light Snow / Windy,Condition_Mostly Cloudy,Condition_Mostly Cloudy / Windy,Condition_Partly Cloudy,Condition_Partly Cloudy / Windy,Condition_Rain,Condition_Rain / Windy,Condition_Snow,Condition_Wintry Mix,Condition_Wintry Mix / Windy
0,11,1,5,-1,124,636,324,323,448,48,34,58,25,38,29.86,9,17,14,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,11,1,5,-7,371,2475,340,333,531,48,34,58,25,38,29.86,9,17,15,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,11,1,5,40,181,1069,301,341,482,48,34,58,25,38,29.86,9,17,22,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,11,1,5,-2,168,944,345,343,513,48,34,58,25,38,29.86,9,17,12,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,11,1,5,-4,139,760,360,356,499,46,32,58,24,35,29.91,9,17,13,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#Regression

In [31]:
X = df.drop(columns=['TAXI_OUT'])
Y = df[['TAXI_OUT']]

In [32]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2,random_state=2021)

In [33]:
len(x_train), len(y_train), len(x_test), len(y_test)

(23056, 23056, 5764, 5764)

In [34]:
## Create a model object

from sklearn.linear_model import LinearRegression
mlr = LinearRegression()

## Fit that object on the training dataset

model_one_hot = mlr.fit(x_train, y_train)
model_one_hot
print(model_one_hot.score(x_train,y_train))

0.15663169127088816


In [35]:
y_test['Predicted_taxi_out'] = model_one_hot.predict(x_test)
y_test.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,TAXI_OUT,Predicted_taxi_out
27614,32,22.405665
10689,20,19.720319
16258,15,21.186167
11608,17,20.767149
23466,16,19.878160


In [36]:
y_test['Error'] = y_test['TAXI_OUT'] - y_test['Predicted_taxi_out']
y_test.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,TAXI_OUT,Predicted_taxi_out,Error
27614,32,22.405665,9.594335
10689,20,19.720319,0.279681
16258,15,21.186167,-6.186167
11608,17,20.767149,-3.767149
23466,16,19.878160,-3.878160


In [37]:
y_test['Squared Error'] = y_test['Error'] ** 2 
y_test.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,TAXI_OUT,Predicted_taxi_out,Error,Squared Error
27614,32,22.405665,9.594335,92.051264
10689,20,19.720319,0.279681,0.078222
16258,15,21.186167,-6.186167,38.268659
11608,17,20.767149,-3.767149,14.191408
23466,16,19.878160,-3.878160,15.040122


In [38]:
square_mean1 = y_test['Squared Error'].mean()
square_mean1

41.10951519192836

In [39]:
import math
RMSE1 = math.sqrt(square_mean1)
RMSE1

6.411670234184565

#Random Forest

In [52]:
from sklearn.ensemble import RandomForestRegressor
#Random Forest
rf_one_hot = RandomForestRegressor(n_estimators=200,random_state=2021)
model_rf_one_hot = rf_one_hot.fit(x_train,y_train)

y_test['Predict_RF'] = model_rf_one_hot.predict(x_test)
print(model_rf_one_hot.score(x_train,y_train))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0.9063369484306686


In [53]:
y_test.head()

,TAXI_OUT,Predicted_taxi_out,Error,Squared Error,Predict_RF,Error_RF,Squared Error_RF
27614,32,22.405665,9.594335,92.051264,23.510,7.675,58.905625
10689,20,19.720319,0.279681,0.078222,21.920,-2.460,6.051600
16258,15,21.186167,-6.186167,38.268659,15.540,-0.705,0.497025
11608,17,20.767149,-3.767149,14.191408,19.660,-2.595,6.734025
23466,16,19.878160,-3.878160,15.040122,20.515,-3.530,12.460900


In [54]:
y_test['Error_RF'] = y_test['TAXI_OUT'] - y_test['Predict_RF']
y_test.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,TAXI_OUT,Predicted_taxi_out,Error,Squared Error,Predict_RF,Error_RF,Squared Error_RF
27614,32,22.405665,9.594335,92.051264,23.510,8.490,58.905625
10689,20,19.720319,0.279681,0.078222,21.920,-1.920,6.051600
16258,15,21.186167,-6.186167,38.268659,15.540,-0.540,0.497025
11608,17,20.767149,-3.767149,14.191408,19.660,-2.660,6.734025
23466,16,19.878160,-3.878160,15.040122,20.515,-4.515,12.460900


In [55]:
y_test['Squared Error_RF'] = y_test['Error_RF'] ** 2 
y_test.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,TAXI_OUT,Predicted_taxi_out,Error,Squared Error,Predict_RF,Error_RF,Squared Error_RF
27614,32,22.405665,9.594335,92.051264,23.51,8.49,72.0801
10689,20,19.720319,0.279681,0.078222,21.92,-1.92,3.6864


In [56]:
square_mean2 = y_test['Squared Error_RF'].mean()
square_mean2

32.256781471200675

In [57]:
import math
RMSE2 = math.sqrt(square_mean2)
RMSE2

5.6795053896620855

#Final Result 

Regression 



1.  R-square : 0.15663169127088816
2.  RMSE : 6.411670234184565


Random Forest
1.  R-square : 0.9063369484306686
2.  RMSE : 5.6795053896620855
